# Aperiodic parameters of the fMRI power spectrum associate with preterm birth and neonatal age 

Machine learning regression analysis predicting neonatal postmenstrual and postnatal age from aperiodic offset and exponent parameters of the resting-state fMRI BOLD signal

This code is developed for the data analysis of the Developing human connectome project dataset (dHCP, third data release) [https://doi.org/10.3389/fnins.2022.886772]. Information about the dataset, including access and terms of use can be found via the following link: https://biomedia.github.io/dHCP-release-notes/

contact: ilksuu@utu.fi

04/12/2025

### Load modules

In [1]:
# remove uninformative and distracting warnings
import warnings
warnings.filterwarnings("ignore")

import sys
# data structures
import pandas as pd
import numpy as np
# scikit-learn
import sklearn as sklearn
from sklearn.model_selection import cross_validate, RepeatedKFold
from sklearn.linear_model import ElasticNetCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
# FOOOF
import fooof
from fooof import FOOOF
# Neurokit2
import neurokit2 as nk2
from neurokit2.signal import signal_psd
# miscellanious
import camelot
from itertools import product
import matplotlib
from matplotlib import pyplot as plt

# check versions
print("Python version = {}".format(sys.version))
print("Numpy version = {}".format(np.__version__))
print("Pandas version = {}".format(pd.__version__))
print("Scikit-learn version = {}".format(sklearn.__version__))
print("Neurokit2 version = {}".format(nk2.__version__))
print("Matplotlib version = {}".format(matplotlib.__version__))
print("FOOOF version = {}".format(fooof.__version__))

Python version = 3.8.20 (default, Oct  3 2024, 15:19:54) [MSC v.1929 64 bit (AMD64)]
Numpy version = 1.23.5
Pandas version = 2.0.3
Scikit-learn version = 1.2.2
Neurokit2 version = 0.2.7
Matplotlib version = 3.7.5
FOOOF version = 1.1.0


### Load demographical data, drop rows based on failed QC etc. criteria, derive target variables

In [2]:
# load demographical data
data = pd.read_csv("documents-export-2022-09-20/demographics-public-data/dHCP-demographics-rel3.csv", sep=";")

In [3]:
# drop bad scans?

drop_qc_flagged = True

if drop_qc_flagged:
    data = data[data["qc_fmri_flagged"] == False]
    
data.shape

(782, 39)

In [4]:
# drop non-singletons?

drop_non_singleton = True

if drop_non_singleton:
    data = data[data["singleton"] == "S"]
    
data.shape

(663, 39)

In [5]:
# use only one scan per subject

data = data.sort_values('scan_number', ascending=True) # sort by scan number for dropping duplicates
data = data.drop_duplicates(subset=['participant_id'], keep="last") # keep only the last scan

data.shape

(605, 39)

In [6]:
# derive and rename key variables

data["postnatal_age"] = data["scan_age"] - data["birth_age"]
data.rename(columns={"scan_age": "postmenstrual_age"}, inplace=True)

### Load ROI names, initialize columns, load MRI data and compute FOOOF-parameters

In [7]:
# load AAL90 atlas region of interest names to a list from a PDF

tables = camelot.read_pdf('documents-export-2022-09-20/src-AAL-brainsci-10-00319-s001.pdf',
                          pages='1,2,3,4,5', flavor="stream")

roi_names = []
for i in range(0,5):
    for rn in list(tables[i].df.iloc[:, 2]):
        roi_names.append(rn)
        
roi_names = roi_names[3:93] # from precentral_L to temporal_inf_R

In [8]:
# create empty columns for offset and exponent parameters (and goodness of fit) for each ROI

roi_offsets = [rn + "_offset" for rn in roi_names]
roi_exponents = [rn + "_exponent" for rn in roi_names]
roi_rsquareds = [rn + "_R^2" for rn in roi_names]

data = data.reindex(data.columns.tolist() + roi_offsets, axis=1)
data = data.reindex(data.columns.tolist() + roi_exponents, axis=1)
data = data.reindex(data.columns.tolist() + roi_rsquareds, axis=1)

# data

In [9]:
# change directory if needed
d = "documents-export-2022-09-20/DATA_dHCP_N719"

# get sampling frequency
tr = 0.392
fs = 1.0/tr

# specify parameters for PSD estimation
method = "multitaper"
normalize = False

# specify parameters for FOOOF estimation
peak_width_limits = (0.001, 12.0) # set min, max peak width
min_freq = 0.01
max_freq = 0.15
freq_range = [min_freq, max_freq]
peak_threshold = 1.5
max_n_peaks = 2

# collect ids for failed model fits
failed = []

# print counter for progress
verbose=False

counter = 1
# iterate over rows (i.e. subjects) in the dataframe
for index, row in data.iterrows():
    if verbose:
        print(counter)
        counter += 1
    # get filename/path as subject identifier with pre-/suffix 
    filename = "sub-" + row["participant_id"] + ".txt"
    path = d + "/" + filename
    try:
        f = np.genfromtxt(path, delimiter="  ")
    except:
        print("unable to read file {}".format(filename))
        failed.append(index)
        continue
    # iterate over ROI specific time series in a file
    for ts_idx, ts in enumerate(f):
        # get current ROI name from the list using current index
        roi_name = roi_names[ts_idx]
        # compute PSD using selected method
        psd = signal_psd(ts, sampling_rate=fs, method=method, min_frequency=min_freq,
                         max_frequency=max_freq, normalize=normalize, show=False)
        freqs = psd["Frequency"].values
        power = psd["Power"].values
        # init. FOOOF object and fit
        fm = FOOOF(verbose=False, peak_width_limits=peak_width_limits,
                   max_n_peaks = max_n_peaks, peak_threshold=peak_threshold)
        fm.fit(freqs, power, freq_range)
        # save aperiodic params to corresponding variable in current DataFrame row
        data.at[index, roi_name + "_offset"] = fm.aperiodic_params_[0]
        data.at[index, roi_name + "_exponent"] = fm.aperiodic_params_[1]
        # save R^2 as the metric for goodness of fit
        data.at[index, roi_name + "_R^2"] = fm.r_squared_

unable to read file sub-CC00666XX15.txt
unable to read file sub-CC00735XX18.txt
unable to read file sub-CC00555XX11.txt
unable to read file sub-CC00834XX18.txt
unable to read file sub-CC00801XX09.txt
unable to read file sub-CC00307XX10.txt


In [10]:
print("Number of rows with NaN values before excluding = {}".format(data.filter(regex="exponent").isna().any(axis=1).sum()))
data.drop(failed, inplace=True)
print("Number of rows with NaN values after excluding = {}".format(data.filter(regex="exponent").isna().any(axis=1).sum()))

Number of rows with NaN values before excluding = 6
Number of rows with NaN values after excluding = 0


In [11]:
# analysis pipeline definition

# z-score normalization
scaler = StandardScaler()
# define model as elasticnet with hyperparameter search. fix random seed
model =  ElasticNetCV(l1_ratio=[.1, .5, .75, .9, .95, .99, 1], alphas=[2**x for x in range(-16,16)], cv=5, random_state=42)

# define pipeline with scaler and elasticnetCV
pipe = Pipeline([("scaler", scaler), ("predictor", model)])

# define cv parameters (folds, n. of repeats)
n_folds = 10 # 10-fold CV
n_reps = 10 # 10 repeats

# init. CV object. fix random seed
cv = RepeatedKFold(n_splits=n_folds, n_repeats=n_reps, random_state=42)
# define evaluation metrics
metrics = ["r2", "neg_mean_absolute_error"]

# define a function to display CV results in a pandas dataframe for easier reading
def make_result_df(cv_res):
        
    # define evaluation metrics
    metrics = ["r2", "neg_mean_absolute_error"]
    # define statistical functions
    statfuncs = [np.mean, np.std]
    # define cross-validation fold names
    folds = ["train", "test"]
    
    # get names for statistical functions
    statfunc_names = [s.__name__ for s in statfuncs]
    # make column names of type "train_mean" etc. using product
    table_cols = [p[0] + "_" + p[1] for p in list(product(folds, statfunc_names))]
    # make index names from cv metrics
    table_idx = metrics
    # init. empty dataframe with cols and idx defined above
    result_df = pd.DataFrame(columns=table_cols, index=table_idx)
    # nested loop to fill a dataframe
    for f in folds:
        for m in result_df.index:
            # k for key to get cross-validation results from cv result dictionary
            k = f + "_" + m
            for s in statfuncs:
                # compute mean/std for train/test result
                stat = s(cv_res[k])
                # fill into previously empty df
                result_df.loc[m][f + "_" + s.__name__] = stat
                
    return result_df

### Analyses 1-4 on offset predictors

In [12]:
# analysis 1: offset, predicting postnatal age, full-term participants

result_offset_postnatal_fullterm = cross_validate(clone(pipe), # clone the model
                                                    data[data["birth_age"] >= 37.0].filter(regex="offset"), # X
                                                    data[data["birth_age"] >= 37.0]["postnatal_age"], # y
                                                    cv=cv, scoring=metrics,
                                                    return_train_score=True, return_estimator=True)

make_result_df(result_offset_postnatal_fullterm)

,train_mean,train_std,test_mean,test_std
r2,0.502028,0.02419,0.361451,0.087482
neg_mean_absolute_error,-0.739477,0.019369,-0.825882,0.070973


In [13]:
# analysis 2: offset, predicting postnatal age, all participants

result_offset_postnatal_all = cross_validate(clone(pipe), # clone the model
                                                data.filter(regex="offset"), # X
                                                data["postnatal_age"], # y
                                                cv=cv, scoring=metrics,
                                                return_train_score=True, return_estimator=True)

make_result_df(result_offset_postnatal_all)

,train_mean,train_std,test_mean,test_std
r2,0.309208,0.011121,0.23841,0.116887
neg_mean_absolute_error,-1.828799,0.037898,-1.884743,0.256272


In [14]:
# analysis 3: offset, predicting postmenstrual age, full-term participants

result_offset_postmenstr_fullterm = cross_validate(clone(pipe), # clone the model
                                                    data[data["birth_age"] >= 37.0].filter(regex="offset"), # X
                                                    data[data["birth_age"] >= 37.0]["postmenstrual_age"], # y
                                                    cv=cv, scoring=metrics,
                                                    return_train_score=True, return_estimator=True)

make_result_df(result_offset_postmenstr_fullterm)

,train_mean,train_std,test_mean,test_std
r2,0.474437,0.024171,0.320834,0.105445
neg_mean_absolute_error,-0.955773,0.023607,-1.073965,0.111146


In [15]:
# analysis 4: offset, predicting postmenstrual age, all participants

result_offset_postmenstr_all = cross_validate(clone(pipe), # clone the model
                                                data.filter(regex="offset"), # X
                                                data["postmenstrual_age"], # y
                                                cv=cv, scoring=metrics,
                                                return_train_score=True, return_estimator=True)

make_result_df(result_offset_postmenstr_all)

,train_mean,train_std,test_mean,test_std
r2,0.498799,0.017736,0.349424,0.133172
neg_mean_absolute_error,-1.289291,0.026877,-1.418903,0.150981


### Analyses 5-8 on exponent predictors

In [16]:
# analysis 5: exponent, predicting postnatal age, full-term participants

result_exponent_postnatal_fullterm = cross_validate(clone(pipe), # clone the model
                                                    data[data["birth_age"] >= 37.0].filter(regex="exponent"), # X
                                                    data[data["birth_age"] >= 37.0]["postnatal_age"], # y
                                                    cv=cv, scoring=metrics,
                                                    return_train_score=True, return_estimator=True)

make_result_df(result_exponent_postnatal_fullterm)

,train_mean,train_std,test_mean,test_std
r2,0.478095,0.023249,0.356391,0.102054
neg_mean_absolute_error,-0.75155,0.017447,-0.821675,0.073077


In [17]:
# analysis 6: exponent, predicting postnatal age, all participants

result_exponent_postnatal_all = cross_validate(clone(pipe), # clone the model
                                                data.filter(regex="exponent"), # X
                                                data["postnatal_age"], # y
                                                cv=cv, scoring=metrics, 
                                                return_train_score=True, return_estimator=True)


make_result_df(result_exponent_postnatal_all)

,train_mean,train_std,test_mean,test_std
r2,0.298857,0.013419,0.224915,0.120825
neg_mean_absolute_error,-1.82284,0.040052,-1.885769,0.261328


In [18]:
# analysis 7: exponent, predicting postmenstrual age, full-term participants

result_exponent_postmenstr_fullterm = cross_validate(clone(pipe), # clone the model
                                                    data[data["birth_age"] >= 37.0].filter(regex="exponent"), # X
                                                    data[data["birth_age"] >= 37.0]["postmenstrual_age"], # y
                                                    cv=cv, scoring=metrics,
                                                    return_train_score=True, return_estimator=True)

make_result_df(result_exponent_postmenstr_fullterm)

,train_mean,train_std,test_mean,test_std
r2,0.435175,0.013406,0.304938,0.120036
neg_mean_absolute_error,-0.983065,0.013368,-1.077448,0.114792


In [19]:
# analysis 8: exponent, predicting postmenstrual age, all participants

result_exponent_postmenstr_all = cross_validate(clone(pipe), # clone the model
                                                data.filter(regex="exponent"), # X
                                                data["postmenstrual_age"], # y
                                                cv=cv, scoring=metrics,
                                                return_train_score=True, return_estimator=True)

make_result_df(result_exponent_postmenstr_all)

,train_mean,train_std,test_mean,test_std
r2,0.386779,0.031333,0.20059,0.141199
neg_mean_absolute_error,-1.374564,0.032641,-1.515302,0.175449


### Analyses 9 - 12: on offset + exponent predictors

In [20]:
# analysis 9: offset and exponent, predicting postnatal age, full-term participants


result_allfeats_postnatal_fullterm = cross_validate(clone(pipe), # clone the model
                                            data[data["birth_age"] >= 37.0].filter(regex="(offset|exponent)"), # X
                                            data[data["birth_age"] >= 37.0]["postnatal_age"], # y
                                            cv=cv, scoring=metrics,
                                            return_train_score=True, return_estimator=True)

make_result_df(result_allfeats_postnatal_fullterm)

,train_mean,train_std,test_mean,test_std
r2,0.574434,0.032504,0.399876,0.090234
neg_mean_absolute_error,-0.676548,0.026612,-0.790558,0.069377


In [21]:
# analysis 10: offset and exponent, predicting postnatal age, all participants

result_allfeats_postnatal_all = cross_validate(clone(pipe), # clone the model
                                                data.filter(regex="(offset|exponent)"), # X
                                                data["postnatal_age"], # y
                                                cv=cv, scoring=metrics,
                                                return_train_score=True, return_estimator=True)

make_result_df(result_allfeats_postnatal_all)

,train_mean,train_std,test_mean,test_std
r2,0.318376,0.009415,0.243049,0.115846
neg_mean_absolute_error,-1.805232,0.03768,-1.868114,0.25545


In [22]:
# analysis 11: offset and exponent, predicting postmenstrual age, full-term participants

result_allfeats_postmenstr_fullterm = cross_validate(clone(pipe), # clone the model
                                            data[data["birth_age"] >= 37.0].filter(regex="(offset|exponent)"), # X
                                            data[data["birth_age"] >= 37.0]["postmenstrual_age"], # y
                                            cv=cv, scoring=metrics,
                                            return_train_score=True, return_estimator=True)

make_result_df(result_allfeats_postmenstr_fullterm)

,train_mean,train_std,test_mean,test_std
r2,0.566282,0.04579,0.353412,0.121414
neg_mean_absolute_error,-0.861598,0.046925,-1.039503,0.109678


In [23]:
# analysis 12: offset and exponent, predicting postmenstrual age, all participants

result_allfeats_postmenstr_all = cross_validate(clone(pipe), # clone the model
                                                data.filter(regex="(offset|exponent)"), # X
                                                data["postmenstrual_age"], # y
                                                cv=cv, scoring=metrics,
                                                return_train_score=True, return_estimator=True)

make_result_df(result_allfeats_postmenstr_all)

,train_mean,train_std,test_mean,test_std
r2,0.666775,0.021429,0.408291,0.14852
neg_mean_absolute_error,-1.044084,0.033976,-1.319757,0.16463
